In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from glow import lenses, time_domain_c, freq_domain_c
from glow import physical_units as phys

## Automatic mode

### Basic

We can initalize both the lens object and the units object (to convert back to physical units) at the same time.

In [ ]:
# redshifts are mandatory for all lenses
zl = 1.5
zs = 3.

# all input quantities are regular floats (no astropy units)
p_lens = {'name':'CIS', 'Mvir_Msun':1e12, 'rc_pc':10}

Psi, units = phys.Lens_Units(zl, zs, p_lens)

print(Psi)

Every lens expects different parameters. At the moment there is way to check what are these parameters, other than looking at the code or the documentation. Some other quantites are also computed along the way, and stored with `astropy` units in the object. 

In [ ]:
display(units.d_eff)
display(units.Mlz)
display(units.Sigma_crit)
display(units.xi0)
display(units.Rvir)

If we have an array of dimensionless times we can convert them to physical units.

In [ ]:
taus = np.geomspace(1e-2, 1e1, 5)

ts = units.tau_to_t(taus)

display(taus)
display(ts)

If we specify some units, we will get back a regular `numpy` array with these units, which can be easier to work with without needing to import `astropy`.

In [ ]:
ts = units.tau_to_t(taus, un='s')
ts

In [ ]:
ts = units.tau_to_t(taus, un='day')
ts

Finally we can convert frequencies

In [ ]:
ws = np.geomspace(1e-2, 1e2, 4)

fs = units.w_to_f(ws)
fs

and distances

In [ ]:
xs = np.geomspace(1e-2, 1e2, 4)

xis = units.x_to_xi(xs)
xis

The inverse operations are also implemented. Make sure that either 1) the input has astropy units or 2) you specify the units through the keyword. Example:

In [ ]:
taus = np.geomspace(1e-2, 1e1, 3)
print('Starting:   tau =', taus)

ts = units.tau_to_t(taus, un='hour')
print('t in hours: t   =', ts)

print('-'*30)
print('Revert the transformation:')

## ------------------------------------
taus1 = units.t_to_tau(ts, un='hour')

import astropy.units as u
taus2 = units.t_to_tau(ts*u.hour)
## ------------------------------------

print('  *    ok:  tau =', taus1)
print('  *    ok:  tau =', taus2)

### Different prescriptions

For some lenses we may want to use different unit conventions. We can add them as a new prescription in the `Units` object. For instance:

In [ ]:
zl = 1.3
zs = 3
p_lens = {'name':'NFW', 'Mvir_Msun':1e14}

Psi1, units1 = phys.Lens_Units(zl, zs, p_lens)
print(Psi1.p_phys)
print('')

Psi2, units2 = phys.Lens_Units(zl, zs, p_lens, prescription='radius')
print(Psi2.p_phys)

In the first case, $\psi_0$ has been set to one, while in the second one we are using the scale radius as $\xi_0$. If we want to compare them directly, we must first use the same impact parameter

In [ ]:
# in the units system 1
y1 = 0.4

y_phys = units1.x_to_xi(y1)
y2 = units2.xi_to_x(y_phys)

y2

and then switch to physical time.

In [ ]:
It1 = time_domain_c.It_SingleIntegral_C(Psi1, y1)
It2 = time_domain_c.It_SingleIntegral_C(Psi2, y2)
## --------------------------------------------------

un = 'day'

t1 = units1.tau_to_t(It1.t_grid, un=un)
t2 = units2.tau_to_t(It2.t_grid, un=un)

fig, ax = plt.subplots()
ax.plot(t1, It1.It_grid/2/np.pi, label='default')
ax.plot(t2, It2.It_grid/2/np.pi, label='radius')

ax.set_xscale('log')
ax.set_xlabel('$t$ [%s]' % un)
ax.set_ylabel('$I(t)$')
ax.legend();

### Relating two lenses

We can also use an external unit convention. This is especially useful to relate different lenses.

In [ ]:
zl = 1.2
zs = 3.

p_lens1 = {'name':'CIS', 'Mvir_Msun':1e10, 'rc_pc':20}
p_lens2 = {'name':'SIS', 'Mvir_Msun':1e12}

We can use the first lens as reference for the second one

In [ ]:
Psi1, units = phys.Lens_Units(zl, zs, p_lens1)
Psi2, units = phys.Lens_Units(zl, zs, p_lens2, prescription='external', units_ext=units)

print(Psi1.p_phys)
print(Psi2.p_phys)

or the other way around

In [ ]:
Psi2, units = phys.Lens_Units(zl, zs, p_lens2)
Psi1, units = phys.Lens_Units(zl, zs, p_lens1, prescription='external', units_ext=units)

print(Psi2.p_phys)
print(Psi1.p_phys)

### Composite lenses

We can also easily build composite lenses starting with their physical parameters.

In [ ]:
zl = 0.2
zs = 0.4

p0 = {'name':'point lens', 'M_Msun':2e2}
p1 = {'name':'off-center ball', 'M_Msun':34, 'R_Rsun':100, 'x1_pc':0.2,   'x2_pc':-0.45}
p2 = {'name':'off-center ball',       'M_Msun':8,  'R_Rsun':20,  'x1_pc':-0.15, 'x2_pc':-0.1}

p_lens = [p0, p1, p2]

Psi, units = phys.Lens_Units(zl, zs, p_lens)

for l in Psi.p_phys['lenses']:
    print(l.p_phys)

The first lens in the list will be used to set the units.

In [ ]:
p_lens = [p1, p0, p2]

Psi, units = phys.Lens_Units(zl, zs, p_lens)

for l in Psi.p_phys['lenses']:
    print(l.p_phys)

## Manual mode

### Units from $M_{lz}$

It is possible to compute the conversion of units by directly specifying $M_{lz}$

In [ ]:
Mlz_Msun = 1e8

units = phys.Units_from_Mlz(Mlz_Msun)

Keep in mind that in this case we can convert times and frequencies, but nothing related to physical distances.

In [ ]:
taus = np.geomspace(1e-2, 1e2, 4)

ts = units.tau_to_t(taus)
ts

### Units only

If we know what we are doing, we can bypass initializing the lens and get only the units

In [ ]:
zl = 1.3
zs = 3

p_lens = {'name':'SIS', 'Mvir_Msun':1e10}

units = phys.Units(zl, zs, p_lens)

In many cases, this is enough since the actual computation in dimensionless can be reused for all the redshifts and masses.